In [1]:
# Import necessary libraries
import numpy as np
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from collections import defaultdict
from textwrap import wrap

# Torch ML libraries
import transformers
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import OneCycleLR

In [2]:
# Set intial variables and constants
%config InlineBackend.figure_format='retina'

# Graph Designs
sns.set(style='whitegrid', palette='muted', font_scale=1.2)
HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]
sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))
rcParams['figure.figsize'] = 12, 8

# Random seed for reproducibilty
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
torch.cuda.empty_cache()

# Set GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

device

device(type='cuda')

In [3]:
# Set the model name
model_name = 'prajjwal1/bert-tiny'

# Build a BERT based tokenizer
tokenizer = BertTokenizer.from_pretrained(model_name, force_download=True)

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [4]:
max_tweets = 50
max_len = 100
num_lstms = 5
batch_size = 1 # this is because its used for so much, and really the input will only be one example so it just makes things easier

In [110]:
# Build the Sentiment Classifier class
class SentimentClassifier(nn.Module):

    # Constructor class
    def __init__(self, n_classes):
        super(SentimentClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(model_name)
        self.drop = nn.Dropout(p=0.3)

        self.lstm = [nn.LSTM(input_size=128, hidden_size=50, num_layers=5, batch_first=True).to(device) for i in range(num_lstms)]
        self.batch = torch.nn.BatchNorm1d(self.lstm[0].hidden_size * num_lstms)
        self.out = nn.Sequential(nn.Linear(self.lstm[0].hidden_size*num_lstms, 1), nn.Sigmoid())
        

    # Forward propagation class
    def forward(self, input_ids, attention_mask):
        pooled_output = torch.zeros((batch_size, max_tweets, 128), dtype=torch.float32).to(device)
        print(input_ids.shape)
        for x in range(0 , len(input_ids)):
            _, output = self.bert(input_ids=input_ids[x], attention_mask=attention_mask[x], return_dict=False)
            output = output.to(device)
            pooled_output[:, x].copy_(output)
        
        pooled_output = pooled_output.view(batch_size, num_lstms, max_tweets//num_lstms, 128)

        lstmOut = torch.zeros((batch_size, num_lstms, self.lstm[0].hidden_size), dtype=torch.float32).to(device)
        
        for i in range(len(self.lstm)):
            lstmTempOut, _ = self.lstm[i](pooled_output[:, i, :])
            lstmOut[:, i, :] = lstmTempOut[:, -1, :]
        lstmOut = self.drop(lstmOut)
        lstmOut = lstmOut.view(batch_size, num_lstms*self.lstm[0].hidden_size)
        norm = self.batch(lstmOut)
        return self.out(norm)

In [111]:
model = SentimentClassifier(len(['not depressed', 'depressed']))
model.load_state_dict(torch.load("WEBEATTHEPAPER.pt", map_location=torch.device('cuda')))
model = model.to(device)

/tmp/ipykernel_282991/718773389.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("WEBEATTHEPAPER.pt", map_location=torch.device('cuda')))

In [112]:
def encodeArray(strArr):
    encoding = [
            tokenizer.encode_plus(
                x,
                add_special_tokens=True,
                max_length=max_len,
                return_token_type_ids=False,
                padding = 'max_length',
                truncation = True,
                return_tensors='pt',
            ) for x in strArr
        ]

    return {
        'review_text': strArr,
        'input_ids': [x['input_ids'].flatten() for x in encoding],
        'attention_mask': [x['attention_mask'].flatten() for x in encoding],

    }
#print(encodeArray(["hi", "Hey look buddy, I'm a Warhammer player. That means I solve problems, not problems like"]))

In [115]:
def returnScore(input):
    while len(input) < max_tweets:
        input.append("")
    
    d = encodeArray(input)
    
    input_ids = torch.tensor(np.array([d["input_ids"]])).to(device)
    attention_mask = torch.tensor(np.array([d["attention_mask"]])).to(device)
    
    input = torch.zeros((max_tweets, batch_size, max_len)).to(device)
    for x in range(max_tweets):
        input[x].copy_(input_ids[:, x])

    attention = torch.zeros((max_tweets, batch_size, max_len)).to(device)
    for x in range(max_tweets):
        attention[x].copy_(attention_mask[:, x])
        
    model.eval()
    return model.forward(input.to(torch.int64), attention.to(torch.int64))

In [116]:
print(returnScore(["hi", "Hey look buddy, I'm a Warhammer player. That means I solve problems, not problems like"]))

torch.Size([50, 1, 100])
tensor([[0.4405]], device='cuda:0', grad_fn=<SigmoidBackward0>)
